<h3>Imports and Setup</h3>

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
big_table = pd.read_csv('data/T_T100_SEGMENT_ALL_CARRIER.csv')
area_codes_table = pd.read_csv('data/L_WORLD_AREA_CODES.csv')
carrier_classes = pd.read_csv('data/L_CARRIER_GROUP_NEW.csv')

In [4]:
# Replace World Area Codes (WAC) with descriptions from lookup table
big_table = big_table.merge(area_codes_table, left_on='DEST_WAC', right_on='Code', how='left')
big_table['DEST_WAC'] = big_table['Description']
big_table['ORIGIN_WAC'] = big_table['Description']
big_table = big_table.drop(columns=['Code', 'Description'])

In [5]:
big_table

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP_NEW,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,1,1,1,F,DU
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,3,7,1,F,DU
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,1,1,1,F,DU
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,3,7,1,F,DU
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,6,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,Massachusetts,32250,HYA,"Hyannis, MA",Massachusetts,United States,Massachusetts,1,125,1,2024,1,1,1,F,DU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314502,891.0,889.0,2367000.0,8002.0,4689.0,5250.0,0.0,59.0,33895.0,26497.0,GV,Grant Aviation,D,GV,Grant Aviation,5,31555,ENA,"Kenai, AK",Alaska,United States,Alaska,30299,ANC,"Anchorage, AK",Alaska,United States,Alaska,4,416,1,2024,2,6,1,F,DU
314503,995.0,994.0,2737200.0,8946.0,7247.0,10718.0,0.0,59.0,37921.0,27463.0,GV,Grant Aviation,D,GV,Grant Aviation,5,30299,ANC,"Anchorage, AK",Alaska,United States,Alaska,31555,ENA,"Kenai, AK",Alaska,United States,Alaska,4,416,1,2024,3,7,1,F,DU
314504,999.0,998.0,2748200.0,8991.0,6631.0,5663.0,0.0,59.0,39218.0,30137.0,GV,Grant Aviation,D,GV,Grant Aviation,5,31555,ENA,"Kenai, AK",Alaska,United States,Alaska,30299,ANC,"Anchorage, AK",Alaska,United States,Alaska,4,416,1,2024,3,7,1,F,DU
314505,1522.0,1522.0,1826400.0,8026.0,5084.0,10340.0,0.0,11.0,17768.0,13202.0,S2,Star Marianas Air Inc.,D,S2,Star Marianas Air Inc.,6,35215,TIQ,"Tinian, TT",U.S. Pacific Trust Territories and Possessions,United States,U.S. Pacific Trust Territories and Possessions,34955,SPN,"Saipan, TT",U.S. Pacific Trust Territories and Possessions,United States,U.S. Pacific Trust Territories and Possessions,0,79,1,2024,3,7,1,F,DU


<h3>Filter Major Carriers</h3>

In [6]:
carrier_classes

,Code,Description
0,0,Foreign Carriers
1,1,Large Regional Carriers (carriers with annual ...
2,2,National Carriers (carriers with annual revenu...
3,3,Major Carriers (carriers with annual revenue o...
4,4,Medium Regional Carriers (carriers with annual...
5,5,Small Certificated Carriers (carrier holding c...
6,6,Commuter Carriers (air taxi operator which per...
7,7,All Cargo Carriers operating under cerificates...
8,9,Commuter Carriers (Air Taxi providing Essentia...


In [7]:
# Filter only major carriers (TranStats category 3)
major_carriers = big_table.loc[big_table['CARRIER_GROUP_NEW'] == 3]
major_carriers['CARRIER_NAME'].unique()

array(['Delta Air Lines Inc.', 'USA Jet Airlines Inc.',
       'United Air Lines Inc.', 'Republic Airline', 'JetBlue Airways',
       'Hawaiian Airlines Inc.', 'Allegiant Air', 'Alaska Airlines Inc.',
       'Southwest Airlines Co.', 'American Airlines Inc.',
       'Sun Country Airlines d/b/a MN Airlines', 'Atlas Air Inc.',
       'Federal Express Corporation', 'Polar Air Cargo Airways',
       'Kalitta Air LLC', 'Spirit Air Lines', 'SkyWest Airlines Inc.',
       'Envoy Air', 'Frontier Airlines Inc.', 'United Parcel Service'],
      dtype=object)

In [8]:
# Filter passenger flights only
major_passenger_flights = major_carriers.loc[major_carriers['PASSENGERS'] > 0]
major_passenger_flights['CARRIER_NAME'].unique()

array(['United Air Lines Inc.', 'Republic Airline', 'JetBlue Airways',
       'Delta Air Lines Inc.', 'Hawaiian Airlines Inc.', 'Allegiant Air',
       'Alaska Airlines Inc.', 'Southwest Airlines Co.',
       'American Airlines Inc.', 'Sun Country Airlines d/b/a MN Airlines',
       'Atlas Air Inc.', 'SkyWest Airlines Inc.', 'Envoy Air',
       'Spirit Air Lines', 'Frontier Airlines Inc.'], dtype=object)

In [9]:
# Filter flights performed
major_passenger_flights = major_passenger_flights[major_passenger_flights['DEPARTURES_PERFORMED'] > 10]

<h3>Sort Domestic & International Flights</h3>

In [10]:
# Helper function to return only domestic or only international flights
def filter_international_flights(df, dom_only=False, int_only=False):
    if dom_only == True:
        df = df[(df['ORIGIN_COUNTRY_NAME'] == 'United States') & (df['DEST_COUNTRY_NAME'] == 'United States')]
    if int_only == True:
        df = df[(df['ORIGIN_COUNTRY_NAME'] != 'United States') | (df['DEST_COUNTRY_NAME'] != 'United States')]
    return df

In [11]:
# Filter domestic routes
dom_flights = filter_international_flights(major_passenger_flights, True)
# Filter international routes
int_flights = filter_international_flights(major_passenger_flights, False, True)

<h3>Load Factor Analysis</h3>

In [15]:
# Add load factor metric
major_passenger_flights['LF'] = major_passenger_flights['PASSENGERS']/major_passenger_flights['SEATS']
dom_flights['LF'] = dom_flights['PASSENGERS']/dom_flights['SEATS']
int_flights['LF'] = int_flights['PASSENGERS']/int_flights['SEATS']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


<h6>Best LF overall</h6>

In [13]:
# Select and reorder relevant columns
mpf_abbrv = major_passenger_flights.loc[:, ['ORIGIN', 'DEST', 'CARRIER_NAME', 'DEPARTURES_PERFORMED', 'SEATS', 'PASSENGERS', 'LF', 'DISTANCE']]
# Sort individual carrier routes by load factor
mpf_abbrv.sort_values(by='LF', ascending=False)

,ORIGIN,DEST,CARRIER_NAME,DEPARTURES_PERFORMED,SEATS,PASSENGERS,LF,DISTANCE
196194,BWI,IAH,Spirit Air Lines,12.0,2184.0,2194.0,1.004579,1235.0
192571,BDL,CLT,American Airlines Inc.,11.0,1408.0,1403.0,0.996449,644.0
203111,LAS,BUF,Southwest Airlines Co.,14.0,2002.0,1990.0,0.994006,1986.0
197289,PUJ,BWI,Southwest Airlines Co.,12.0,2100.0,2082.0,0.991429,1504.0
195573,FCA,ORD,United Air Lines Inc.,12.0,1512.0,1498.0,0.990741,1352.0
...,...,...,...,...,...,...,...,...
202524,SLC,CDC,SkyWest Airlines Inc.,14.0,700.0,81.0,0.115714,221.0
241527,ORD,MCW,SkyWest Airlines Inc.,27.0,1350.0,153.0,0.113333,288.0
205512,LAX,GJT,SkyWest Airlines Inc.,15.0,1020.0,114.0,0.111765,655.0
223572,MCW,ORD,SkyWest Airlines Inc.,21.0,1050.0,112.0,0.106667,288.0


<h6>Best LF domestic routes</h6>

In [14]:
# Select and order specified columns for domestic flights
mpf_dom_abbrv = dom_flights.loc[:, ['ORIGIN', 'DEST', 'CARRIER_NAME', 'DEPARTURES_PERFORMED', 'SEATS', 'PASSENGERS', 'LF', 'DISTANCE']]
# Sort individual carrier routes by load factor
mpf_dom_abbrv.sort_values(by='LF', ascending=False)
# Lamda function to categorize flights as domestic/international
major_passenger_flights['DOMESTIC'] = major_passenger_flights.apply(lambda row: 'D' if row['ORIGIN_COUNTRY_NAME'] == 'United States' and row['DEST_COUNTRY_NAME'] == 'United States' else 'I', axis=1)

KeyError: "['LF'] not in index"

<h6>Best LF international routes</h6>

In [ ]:
# Select and order specified columns for international flights
mpf_int_abbrv = int_flights.loc[:, ['ORIGIN', 'DEST', 'ORIGIN_CITY_NAME', 'DEST_CITY_NAME','CARRIER_NAME', 'DEPARTURES_PERFORMED', 'SEATS', 'PASSENGERS', 'LF', 'DISTANCE']]
# Sort individual carrier routes by load factor
mpf_int_abbrv.sort_values(by='LF', ascending=False)

<h6>LF by carrier, domestic and international</h6>

In [ ]:
# Select SEATS, and PASSENGERS columns and group by airline and region
seats_by_carrier = major_passenger_flights.groupby(['CARRIER_NAME', 'REGION']).agg({'SEATS': 'sum', 'PASSENGERS': 'sum'}).reset_index()
seats_by_carrier

In [ ]:
# Pivot to create separate columns for each region and metric
pivoted_regions = seats_by_carrier.pivot_table(index='CARRIER_NAME', columns='REGION', values=['SEATS', 'PASSENGERS'], fill_value=0)

# Flatten the MultiIndex in columns and rename columns
pivoted_regions.columns = [f'{metric}_{region}' for metric, region in pivoted_regions.columns]

# Reset index to turn CARRIER_NAME back into a column
pivoted_regions = pivoted_regions.reset_index()

# Display the resulting table
pivoted_regions

In [ ]:
# Group by CARRIER_NAME and DOMESTIC, then sum SEATS and PASSENGERS
seats_by_carrier = major_passenger_flights.groupby(['CARRIER_NAME', 'DOMESTIC']).agg({
    'SEATS': 'sum',
    'PASSENGERS': 'sum'
}).reset_index()

# Pivot to create separate columns for Domestic and International regions
pivoted_ID_seats_pax = seats_by_carrier.pivot_table(index='CARRIER_NAME', columns='DOMESTIC', values=['SEATS', 'PASSENGERS'], fill_value=0)

# Flatten the MultiIndex in columns and rename columns
pivoted_ID_seats_pax.columns = [f'{metric}_{region}' for metric, region in pivoted_ID_seats_pax.columns]

# Reset index to turn CARRIER_NAME back into a column
pivoted_ID_seats_pax = pivoted_ID_seats_pax.reset_index()

pivoted_ID_seats_pax

In [ ]:
# Create LF_by_carrier data frame
LF_by_carrier = pivoted_ID_seats_pax
# Define Total/Domestic/International LF columns
LF_by_carrier['Total LF'] = (pivoted_ID_seats_pax['PASSENGERS_D'] + pivoted_ID_seats_pax['PASSENGERS_I']) / (pivoted_ID_seats_pax['SEATS_D'] + pivoted_ID_seats_pax['SEATS_I'])
LF_by_carrier['Domestic LF'] = pivoted_ID_seats_pax['PASSENGERS_D'] / pivoted_ID_seats_pax['SEATS_D']
LF_by_carrier['International LF'] = pivoted_ID_seats_pax['PASSENGERS_I'] / pivoted_ID_seats_pax['SEATS_I']
# Select applicable columns
LF_by_carrier = LF_by_carrier.loc[:, ['CARRIER_NAME', 'Total LF', 'Domestic LF', 'International LF']]
# Sort by total load factor, highest to lowest
LF_by_carrier.sort_values(by='Total LF', ascending=False)

<h6>Load Factor by Destination, All</h6>

In [ ]:
# Aggregate seats, pax, and departures by origin airport/city name
flights_by_dest = major_passenger_flights.groupby(['DEST', 'DEST_CITY_NAME']).agg({
    'SEATS': 'sum',
    'PASSENGERS': 'sum',
    'DEPARTURES_PERFORMED': 'sum'
    ''
}).reset_index()
# Calculate load factor
flights_by_dest['LF'] = flights_by_dest['PASSENGERS'] / flights_by_dest['SEATS']
# Select columns and sort by load factor (highest to lowest)
flights_by_dest.loc[:,['DEST', 'DEST_CITY_NAME', 'LF','DEPARTURES_PERFORMED']].sort_values(by='LF', ascending=False)

<h6>Load Factor by Destination, Domestic</h6>

In [ ]:
# Repeat analysis with domestic flights
# Aggregate seats, pax, and departures by origin airport/city name
dom_flights_by_dest = dom_flights.groupby(['DEST', 'DEST_CITY_NAME']).agg({
    'SEATS': 'sum',
    'PASSENGERS': 'sum',
    'DEPARTURES_PERFORMED': 'sum'
}).reset_index()
# Calculate load factor
dom_flights_by_dest['LF'] = dom_flights_by_dest['PASSENGERS'] / dom_flights_by_dest['SEATS']
# Select columns and sort by load factor (highest to lowest)
dom_flights_by_dest.loc[:,['DEST', 'DEST_CITY_NAME', 'LF','DEPARTURES_PERFORMED']].sort_values(by='LF', ascending=False)

<h6>Load Factor by Origin, Domestic</h6>

In [ ]:
# Repeat analysis with domestic flights
# Aggregate seats, pax, and departures by origin airport/city name
dom_flights_by_origin = dom_flights.groupby(['ORIGIN', 'ORIGIN_CITY_NAME']).agg({
    'SEATS': 'sum',
    'PASSENGERS': 'sum',
    'DEPARTURES_PERFORMED': 'sum'
}).reset_index()
# Calculate load factor
dom_flights_by_origin['LF'] = dom_flights_by_origin['PASSENGERS'] / dom_flights_by_origin['SEATS']
# Select columns and sort by load factor (highest to lowest)
dom_flights_by_origin.loc[:,['ORIGIN', 'ORIGIN_CITY_NAME', 'LF','DEPARTURES_PERFORMED']].sort_values(by='LF', ascending=False)